In [ ]:
#换页
import dash
import dash_html_components as html
import dash_bootstrap_components as dbc
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# app = dash.Dash(__name__)

app.layout = html.Div(
    dbc.Container(
        dbc.Tabs(
            [
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('这是选项卡1')
                    ],
                    label='选项卡1'
                ),
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('这是选项卡2')
                    ],
                    label='选项卡2'
                ),
                dbc.Tab(
                    [
                        html.Br(),
                        html.P('这是选项卡3')
                    ],
                    label='选项卡3'
                ),
            ]
        ),
        style={'margin-top': '100px'}
    )
)
if __name__ == '__main__':
    app.run_server()
# if __name__ == '__main__':
#     app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2022 11:08:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 11:08:36] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_0m1658298570.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 11:08:37] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 11:08:37] "GET /_dash-dependencies HTTP/1.1" 200 -


In [7]:
%tb


SystemExit: 1

In [1]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc 
import dash_html_components as html

# from pandas_datareader import data as web 
import yfinance as yf
from datetime import datetime as dt

import numpy as np 
import pandas as pd
from ta.trend import MACD
from ta.momentum import StochasticOscillator
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math
print('hello')
app = dash.Dash(__name__,
                external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.layout = html.Div([
    dcc.Dropdown(
        id='my-dropdown',
        options=[
            {'label': 'Coke', 'value': 'COKE'},
            {'label': 'Tesla', 'value': 'TSLA'},
            {'label': 'Apple', 'value': 'AAPL'}
        ],  
        value='TSLA'
    ),  
    dcc.Graph(id='my-graph1'),
    dcc.Graph(id='my-graph2'),
    dcc.Graph(id='my-graph3'),
    dcc.Graph(id='my-graph4'),
    dcc.Graph(id='my-graph5'),
    dcc.Graph(id='my-graph6')
], style={'width': '500'})

@app.callback([Output('my-graph1','figure'),Output('my-graph2','figure'),Output('my-graph3','figure'),Output('my-graph4','figure'),Output('my-graph5','figure'),Output('my-graph6','figure')], [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown_value):

    df =  yf.download(selected_dropdown_value, start='2020-01-01', proxy="127.0.0.1:33210")
    
    fig1 = go.Figure(go.Candlestick(x=df.index,
      open=df['Open'],
      high=df['High'],
      low=df['Low'],
      close=df['Close'],
      showlegend=False),
      layout=go.Layout(title=go.layout.Title(text=f"Stock Price and Moving Average of {selected_dropdown_value}")))

    #把非交易日排除掉
    # hide weekends
    fig1.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])]) 
    # removing all empty dates
    # build complete timeline from start date to end date
    dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
    # retrieve the dates that ARE in the original datset
    dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
    # define dates with missing values
    dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
    fig1.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

    #add moving average
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    fig1.add_trace(go.Scatter(x=df.index, 
                         y=df['MA5'], 
                         opacity=0.7, 
                         line=dict(color='blue', width=2), 
                         name='MA 5'))

    fig1.add_trace(go.Scatter(x=df.index, 
                         y=df['MA20'], 
                         opacity=0.7, 
                         line=dict(color='orange', width=2), 
                         name='MA 20'))
    fig1.add_trace(go.Scatter(x=df.index, 
                         y=df['MA50'], 
                         opacity=0.7, 
                         line=dict(color='pink', width=2), 
                         name='MA 50'))

    # Plot volume trace on 2nd row 
    fig2 = go.Figure(go.Bar(x=df.index, 
                     y=df['Volume'],
                    ),layout=go.Layout(title=go.layout.Title(text=f"Stock Trading Volumn of {selected_dropdown_value}")))
    # Plot MACD trace on 3rd row
    # MACD
    macd = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            window_sign=9)
    fig3 = go.Figure(go.Bar(x=df.index, 
                     y=macd.macd_diff(),
                     name = 'MACD_DIF',
                    ),layout=go.Layout(title=go.layout.Title(text=f"MACD Indicator {selected_dropdown_value}")))
    fig3.add_trace(go.Scatter(x=df.index,
                         y=macd.macd(),
                         line=dict(color='black', width=2),
                         name = 'MACD_DEA'
                        ))
    fig3.add_trace(go.Scatter(x=df.index,
                         y=macd.macd_signal(),
                         line=dict(color='blue', width=1),
                         name = 'MACD moving average of 3 days'
                        ))
    # Plot stochastics trace on 4th row
    # stochastic
    stoch = StochasticOscillator(high=df['High'],
                             close=df['Close'],
                             low=df['Low'],
                             window=14, 
                             smooth_window=3)
    fig4 = go.Figure(go.Scatter(x=df.index,
                         y=stoch.stoch(),
                         line=dict(color='black', width=2),
                         name = 'stochastic'
                        ),layout=go.Layout(title=go.layout.Title(text=f"Stochastic of {selected_dropdown_value}")))
    fig4.add_trace(go.Scatter(x=df.index,
                         y=stoch.stoch_signal(),
                         name = 'stochastic moving average of 3 days',
                         line=dict(color='blue', width=1),
                        ))
    #传统的stochastics oscillator 80超买，20超卖,把这个加进stochastics
    
    const1 = np.linspace(start=20, stop=20,num=len(df.index))
    const2 = np.linspace(start=80, stop=80,num=len(df.index))
    fig4.add_trace(go.Scatter(x=df.index,
                         y=const1,
                         line=dict(color='red',width=1),
                         name = '20'
                        ))
    fig4.add_trace(go.Scatter(x=df.index,
                         y=const2,
                         line=dict(color='green',width=1),
                         name = '80'
                        ))
    
    #fig5 volatility
    df.insert(df.shape[1],'Ln',0)
    df.insert(df.shape[1],'Ln2',0)

    for i in range(df.shape[0]-1):
        df['Ln'].iloc[i+1] = math.log(df['Close'].iloc[i+1]/df['Close'].iloc[i])
        df['Ln2'].iloc[i+1] = df['Ln'].iloc[i+1]**2
    df.insert(df.shape[1],'Volatility',0)
    for i in range(df.shape[0]-2):
        df['Volatility'].iloc[i+2] = math.sqrt(df['Ln2'].iloc[1:i+2].sum()/(i+2-1))
    fig5 = go.Figure(go.Scatter(x=df.index,
                         y=df['Volatility'],
                         line=dict(color='blue',width=1)
                        ),layout=go.Layout(title=go.layout.Title(text=f"Volatility of {selected_dropdown_value}")))
    
    #figure6 value at risk
    df['daily_ret'] = df['Close'].pct_change(1)
    # print(df)
    sRate = df['daily_ret'].iloc[1:].sort_values(ascending=True)
    # print(sRate)
    p = np.percentile(sRate, (1, 5, 10), interpolation='midpoint') # 输出分位度为1%，5%和10%即置信度99%，95%和90%时的值
    print(p) #1%分位值为第一个-0.05377872，即根据历史数据，value at risk回报率高于-0.05377872的可能为99%
    fig6 = go.Figure(go.Bar(x=['1%','5%','10%'],
                     y=p
                     ),layout=go.Layout(title=go.layout.Title(text=f"Value at risk on 1%, 5% and 10% of {selected_dropdown_value}")))
    return fig1, fig2, fig3, fig4, fig5, fig6

if __name__ == '__main__':
    app.run_server()

C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


hello
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [21/Jul/2022 11:05:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 11:05:48] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 11:05:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Jul/2022 11:05:49] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Jul/2022 11:05:49] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [21/Jul/2022 11:05:49] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


[*********************100%***********************]  1 of 1 completed


C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:145: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:146: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:149: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:160: DeprecationWarning:

the 

[-0.12087212 -0.06521483 -0.0470029 ]
[*********************100%***********************]  1 of 1 completed


C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:145: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:146: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:149: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\szf1998\AppData\Local\Temp\ipykernel_5856\482581248.py:160: DeprecationWarning:

the 

[-0.05995236 -0.03431782 -0.02550846]
